# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [ ]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [ ]:
song_files = get_files('data/song_data')

In [ ]:
print(len(song_files),'\n',type(song_files))

In [ ]:
filepath = song_files[0]

In [ ]:
print(filepath)

In [ ]:
file_df=[]

for index,file in enumerate(song_files):
    data = pd.read_json(file, lines=True)

    file_df.append(data)
    
df = pd.concat(file_df, ignore_index=True)
# df = pd.read_json(filepath, lines=True)
print(df.shape)
df.head()

In [ ]:
df.dtypes #checking the type of dataframe to determine the duration data type

In [ ]:
print(len(df.loc[df['artist_name'] == 'Gang Starr']))

In [ ]:
# print(len(df['artist_name'].unique()))
# df['artist_name'].unique()
unique_artist_list=df['artist_name'].unique().tolist()
print(len(unique_artist_list),'\n',unique_artist_list)

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
columns=['song_id','title','artist_id','year','duration']
# song_data =df[columns].values
song_data =df[columns]
song_data.head()
#song_data_values=song_data.values

In [ ]:
print(song_table_insert,'\n',len(song_data))

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
# cur.execute(song_table_insert, song_data)
# conn.commit()
for i, row in song_data.iterrows():
    cur.execute(song_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
columns_artists=['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']
# artist_data = df[columns_artists].values
artist_data = df[columns_artists].fillna(0)
artist_data.head()

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
# print('insert table query is: \n',artist_table_insert)
# cur.execute(artist_table_insert, artist_data[0])
# conn.commit()

for i, row in artist_data.iterrows():
    cur.execute(artist_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files('data/log_data')

In [ ]:
print(len(log_files))

In [ ]:
filepath = log_files[0]

In [ ]:
print(filepath)

In [ ]:
# df = pd.read_json(filepath, lines=True)
# df.head()

file_df_log=[]

for index,file in enumerate(log_files):
    data_log = pd.read_json(file, lines=True)

    file_df_log.append(data_log)
    
df_log = pd.concat(file_df_log, ignore_index=True)
# df = pd.read_json(filepath, lines=True)
print(df_log.shape,'\n',df_log.dtypes)
df_log.head()

In [ ]:
df_test_song = df_log[df_log['artist'] == 'Gold']
df_test_song.head()

In [ ]:
df_log['artist'].unique()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df_log = df_log[df_log['page'] == 'NextSong']
df_log.head()
#df.info()

In [ ]:
print(df_log.shape)
df_log.dtypes

In [ ]:
#df['ts']= pd.to_datetime(df['ts'])
#df.dtypes
t = pd.to_datetime(df_log['ts'],unit='ms')
print(t.shape)
t.head()

In [ ]:
time_data = (t,t.dt.hour,t.dt.day,t.dt.weekofyear,t.dt.month,t.dt.year,t.dt.weekday)
print(type(time_data),'\n',time_data)
column_labels = ('timestamp', 'hour', 'day', 'week of year', 'month', 'year', 'weekday')

In [ ]:
test={
    'name':('ahmed','neha'),
    'age':(32,28),
    'gender':('male','female')
}
time_df = pd.DataFrame(test)
time_df.head()

In [ ]:
test_dict=[]
for index,label in enumerate(column_labels) :
    test_dict.append((column_labels[index],time_data[index]))
test_dict_tupple=tuple(test_dict)
# print(test_dict)
#convert_test=dict(test_dict) testing
time_df_test=pd.DataFrame(dict(test_dict_tupple))
time_df_test.head()

# for x in column_labels:
#     test_dict=((column_labels[column_labels.index(x)],time_data[column_labels.index(x)]))
# print(test_dict)

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
print(time_table_insert)

In [ ]:
for i, row in time_df_test.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
df_log.head()

In [ ]:
user_df = df_log[['userId','firstName','lastName','gender','level']]
print(user_df.shape)
user_df.head()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
#     print(row.dtype)
#     print(len(list(row)),'\n',type(list(row)),'\n',list(row))
#     break
    cur.execute(user_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
song_artist_query="SELECT * FROM songs JOIN artists ON songs.artist_id=artists.artist_id"
try:
    cur.execute(song_artist_query)
    query_record = cur.fetchone()
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)
print(type(query_record),'\n',query_record)

In [ ]:
print(song_select)

In [ ]:
#testing to unpack tuples
t = (0, 1, 2,5,7,8)
a, b, *_ = t

print(a)
print(b)
print(_)



In [ ]:
for index, row in df_log.iterrows():
    
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        #unpack tuples
        songid, artistid,*_ =results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [ ]:
len(songplay_data)

In [ ]:
for index, row in df_log.iterrows():
    print(row)
    break

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.